In [1]:
import os 
import pandas as pd

import ast

from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)

In [4]:
DATA_DIR = os.getenv("DATA_DIR")

In [5]:
clean_content_df = pd.read_csv(os.path.join(DATA_DIR, 'metadata/clean_content_links.csv')
                              )

In [6]:
clean_content_df.shape

(236590, 14)

In [7]:
clean_content_df.head()

,base_path,content_id,document_type,primary_publishing_organisation,publishing_app,title,ordered_related_items,quick_links,related_mainstream_content,related_guides,document_collections,part_of_step_navs,related_to_step_navs,slugs
0,/government/publications/list-of-psychologists...,04a0cc0d-0b9f-45ad-bf57-7c54cbab9df9,guidance,Foreign & Commonwealth Office,whitehall,chile - list of psychologists and psychiatrist...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/government/statistics/uk-labour-market-statis...,a61985b0-d6eb-4cf1-8140-642b9557ce00,national_statistics,Office for National Statistics,whitehall,uk labour market statistics: may 2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/government/publications/monitor-remuneration-...,d569ef4b-d632-49a0-9795-6a7ea934b799,transparency,Monitor,whitehall,monitor: remuneration committee papers october...,NaN,NaN,NaN,NaN,['5f4c7b0f-7631-11e4-a3cb-005056011aef'],NaN,NaN,NaN
3,/government/statistical-data-sets/env26-expend...,5e0fee54-7631-11e4-a3cb-005056011aef,statistical_data_set,"Department for Environment, Food & Rural Affairs",whitehall,env26 - expenditure on biodiversity,NaN,NaN,NaN,NaN,['5eb6d242-7631-11e4-a3cb-005056011aef'],NaN,NaN,NaN
4,/government/publications/hmg-spending-moratori...,581cabaf-2ed1-4411-8d1b-35b1bcb559b5,transparency,Department for International Development,whitehall,hmg spending moratoria: dfid ict january to ma...,NaN,NaN,NaN,NaN,['5eb71ddb-7631-11e4-a3cb-005056011aef'],NaN,NaN,NaN


In [8]:
clean_content_df[
    clean_content_df['related_mainstream_content'].notnull() |
    clean_content_df['ordered_related_items'].notnull() |
    clean_content_df['part_of_step_navs'].notnull()].count()

base_path                          2290
content_id                         2290
document_type                      2290
primary_publishing_organisation    2259
publishing_app                     2290
title                              2290
ordered_related_items              1935
quick_links                           0
related_mainstream_content          312
related_guides                      141
document_collections                134
part_of_step_navs                   170
related_to_step_navs                 57
slugs                               749
dtype: int64

In [9]:
# only select rows that have related links (they have 
# related_mainstream_content, ordered_related_items, or quick_links)
clean_content_rl_df = clean_content_df.copy().query(
    'related_mainstream_content.notnull() or ordered_related_items.notnull()or part_of_step_navs.notnull() or quick_links.notnull()'
    )

In [10]:
clean_content_rl_df.shape

(2290, 14)

In [11]:
# fill NaNs with empty arrays, and then literal_eval all the arrays so we can
# access the items within them (the different slugs associated with each
# content ID)
clean_content_rl_df['slugs'] = clean_content_rl_df['slugs'].fillna("['']").apply(
    ast.literal_eval)

This converts the string to a list and puts an empty list where there are none

In [12]:
clean_content_rl_df.reset_index(inplace=True, drop=True)

In [13]:
clean_content_rl_df['slugs'].head()

0                                                   []
1    [overview, rules-for-class-3-invalid-carriages...
2    [overview, what-you-can-get, eligibility, how-...
3                                                   []
4    [overview, how-it-works, calculating-dea, what...
Name: slugs, dtype: object

In [14]:
clean_content_rl_df['slugs'][0]

['']

In [15]:
clean_content_rl_df['slugs'][1]

['overview',
 'rules-for-class-3-invalid-carriages',
 'driving-on-the-road',
 'driving-on-footpaths-and-parking',
 'eyesight-requirements',
 'use-by-nondisabled-people',
 'vehicle-tax-registration-and-insurance']

In [17]:
# sometimes there isn't an empty slug in the list of slugs, but the page path
# exists, so this is a little hack to includ the plain basePath
def add_dummy_slug(slugs):
    list1 = ['']
    list1.extend(slugs)
    return list(set(list1))

dummmy_slug is the url without any slug which  is also a page so needs an empty slug

In [19]:
clean_content_rl_df['slugs'] = clean_content_rl_df['slugs'].apply(add_dummy_slug)

wide to long

In [20]:
# adapted from https://gist.github.com/jlln/338b4b0b55bd6984f883
def splitDataFrameList(df,target_column):
    '''
    df = dataframe to split,
    target_column = the column containing the values to split, in an array
    returns: a dataframe with each entry for the target column separated,
        with each element moved into a new row. The values in the other
        columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row,row_accumulator,target_column):
        for s in row[target_column]:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column))
    new_df = pd.DataFrame(new_rows)
    return new_df

In [21]:
clean_content_exploded_df = splitDataFrameList(
    clean_content_rl_df, 'slugs')

In [22]:
def add_slash_between_basepath_slug(slug):
    if slug == '':
        return ''
    else:
        return '/' + slug

In [23]:
clean_content_exploded_df['slug'] = clean_content_exploded_df['slugs'].map(
    add_slash_between_basepath_slug)

In [24]:
clean_content_exploded_df['pagePath'] = clean_content_exploded_df['base_path']  + clean_content_exploded_df['slug']


In [25]:
clean_content_exploded_df.head()

,base_path,content_id,document_collections,document_type,ordered_related_items,part_of_step_navs,primary_publishing_organisation,publishing_app,quick_links,related_guides,related_mainstream_content,related_to_step_navs,slugs,title,slug,pagePath
0,/company-voluntary-arrangements,6fafa451-96b4-4d38-a97d-5261c8a62dd9,NaN,answer,"['86e9a768-4162-4d75-9744-f92f56f461ae', 'a0c7...",NaN,Government Digital Service,publisher,NaN,NaN,NaN,NaN,,company voluntary arrangements,,/company-voluntary-arrangements
1,/mobility-scooters-and-powered-wheelchairs-rules,b3202d1a-f26d-49a2-8f2b-0c7923ba410d,NaN,guide,"['fa748fae-3de4-4266-ae85-0797ada3f40c', '2157...",NaN,Government Digital Service,publisher,NaN,NaN,NaN,NaN,,mobility scooters and powered wheelchairs: the...,,/mobility-scooters-and-powered-wheelchairs-rules
2,/mobility-scooters-and-powered-wheelchairs-rules,b3202d1a-f26d-49a2-8f2b-0c7923ba410d,NaN,guide,"['fa748fae-3de4-4266-ae85-0797ada3f40c', '2157...",NaN,Government Digital Service,publisher,NaN,NaN,NaN,NaN,use-by-nondisabled-people,mobility scooters and powered wheelchairs: the...,/use-by-nondisabled-people,/mobility-scooters-and-powered-wheelchairs-rul...
3,/mobility-scooters-and-powered-wheelchairs-rules,b3202d1a-f26d-49a2-8f2b-0c7923ba410d,NaN,guide,"['fa748fae-3de4-4266-ae85-0797ada3f40c', '2157...",NaN,Government Digital Service,publisher,NaN,NaN,NaN,NaN,eyesight-requirements,mobility scooters and powered wheelchairs: the...,/eyesight-requirements,/mobility-scooters-and-powered-wheelchairs-rul...
4,/mobility-scooters-and-powered-wheelchairs-rules,b3202d1a-f26d-49a2-8f2b-0c7923ba410d,NaN,guide,"['fa748fae-3de4-4266-ae85-0797ada3f40c', '2157...",NaN,Government Digital Service,publisher,NaN,NaN,NaN,NaN,overview,mobility scooters and powered wheelchairs: the...,/overview,/mobility-scooters-and-powered-wheelchairs-rul...


In [26]:
clean_content_exploded_df[clean_content_exploded_df['base_path'] == '/deposit-protection-schemes-and-landlords']

,base_path,content_id,document_collections,document_type,ordered_related_items,part_of_step_navs,primary_publishing_organisation,publishing_app,quick_links,related_guides,related_mainstream_content,related_to_step_navs,slugs,title,slug,pagePath
3098,/deposit-protection-schemes-and-landlords,806a3397-878a-492f-977a-a5a2afbf249c,NaN,guide,"['fd824dab-4f80-4a20-a7fd-b48c2c2f70f8', '3c65...",NaN,Government Digital Service,publisher,NaN,NaN,NaN,NaN,,deposit protection schemes and landlords,,/deposit-protection-schemes-and-landlords
3099,/deposit-protection-schemes-and-landlords,806a3397-878a-492f-977a-a5a2afbf249c,NaN,guide,"['fd824dab-4f80-4a20-a7fd-b48c2c2f70f8', '3c65...",NaN,Government Digital Service,publisher,NaN,NaN,NaN,NaN,information-you-must-give-to-your-tenants,deposit protection schemes and landlords,/information-you-must-give-to-your-tenants,/deposit-protection-schemes-and-landlords/info...
3100,/deposit-protection-schemes-and-landlords,806a3397-878a-492f-977a-a5a2afbf249c,NaN,guide,"['fd824dab-4f80-4a20-a7fd-b48c2c2f70f8', '3c65...",NaN,Government Digital Service,publisher,NaN,NaN,NaN,NaN,overview,deposit protection schemes and landlords,/overview,/deposit-protection-schemes-and-landlords/over...
3101,/deposit-protection-schemes-and-landlords,806a3397-878a-492f-977a-a5a2afbf249c,NaN,guide,"['fd824dab-4f80-4a20-a7fd-b48c2c2f70f8', '3c65...",NaN,Government Digital Service,publisher,NaN,NaN,NaN,NaN,if-you-dont-protect-your-tenants-deposit,deposit protection schemes and landlords,/if-you-dont-protect-your-tenants-deposit,/deposit-protection-schemes-and-landlords/if-y...
3102,/deposit-protection-schemes-and-landlords,806a3397-878a-492f-977a-a5a2afbf249c,NaN,guide,"['fd824dab-4f80-4a20-a7fd-b48c2c2f70f8', '3c65...",NaN,Government Digital Service,publisher,NaN,NaN,NaN,NaN,disputes,deposit protection schemes and landlords,/disputes,/deposit-protection-schemes-and-landlords/disp...


In [27]:
clean_content_exploded_df.shape

(5977, 16)

In [28]:
clean_content_exploded_df.drop_duplicates().shape

(5977, 16)

In [29]:
len(set(clean_content_exploded_df['pagePath']))
# we should dedupe the list wehen we read it

5977

In [35]:
clean_content_exploded_df.to_csv(os.path.join(DATA_DIR, 'metadata/loved_pages.csv.gz'),
                        compression='gzip', index=False)

In [33]:
clean_content_exploded_df.groupby('document_type').count()

,base_path,content_id,document_collections,ordered_related_items,part_of_step_navs,primary_publishing_organisation,publishing_app,quick_links,related_guides,related_mainstream_content,related_to_step_navs,slugs,title,slug,pagePath
document_type,,,,,,,,,,,,,,,
answer,694,694,0,691,29,694,694,0,0,0,16,694,694,694,694
calendar,2,2,0,2,0,2,2,0,0,0,0,2,2,2,2
detailed_guide,317,317,122,0,10,317,317,0,141,312,1,317,317,317,317
form,10,10,7,0,10,10,10,0,0,0,0,10,10,10,10
guidance,16,16,5,0,16,16,16,0,0,0,0,16,16,16,16
guide,4436,4436,0,4426,387,4436,4436,0,0,0,198,4436,4436,4436,4436
licence,60,60,0,60,0,60,60,0,0,0,0,60,60,60,60
local_transaction,128,128,0,128,6,128,128,0,0,0,1,128,128,128,128
manual,1,1,0,0,1,1,1,0,0,0,0,1,1,1,1


In [34]:
clean_content_exploded_df[
    clean_content_exploded_df['document_type'] == 'simple_smart_answer'].to_csv(os.path.join(DATA_DIR, 'metadata/loved_smart_answers.csv.gz'),
                        compression='gzip', index=False)

In [36]:
clean_content_df[clean_content_df['content_id'] == 'bb4c660d-3f60-476a-8378-2c1af2e548d1']

,base_path,content_id,document_type,primary_publishing_organisation,publishing_app,title,ordered_related_items,quick_links,related_mainstream_content,related_guides,document_collections,part_of_step_navs,related_to_step_navs,slugs
64327,/government/statistics/primary-school-performa...,bb4c660d-3f60-476a-8378-2c1af2e548d1,national_statistics,Department for Education,whitehall,primary school performance tables: 2018,NaN,NaN,NaN,NaN,"['5efbd793-7631-11e4-a3cb-005056011aef', '5eb7...",NaN,NaN,NaN
